In [2]:
###################################################################################
#                      2. 정형 데이터마이닝 (사용 데이터 : BlackFriday)                  
###################################################################################

#---------------------------------------------------------------------------------------
# Q1) "BlackFriday"데이터에서 Product_Category_2, Product_Category_3의 NA 값에 0값으로 대체하고
#     Product_categry_1, 2, 3 변수의 값을 다 더한 Product_all 변수를 생성하여 추가하라.
#     그리고 User_ID를 character 변수로, Occupation, Marital_Status, Product_Category_1, 
#     Product_Category_2, Product_Category_3 변수를 범주형 변수로 변환하시오. 
#     마지막으로 범주형 변수인 Gender, Age, City_Category, Stay_In_Current_City_Years를 
#     더미변수로 변환해서 BlackFriday 데이터에 추가하시오.
#---------------------------------------------------------------------------------------

import pandas as pd
import numpy as np

df = pd.read_csv('./data/모의고사3회/BlackFriday.csv')
# print(df.describe())
# print(df.head())

# Product_Category_2, Product_Category_3의 NA 값에 0값으로 대체
# 데이터 프레임 내의 컬럼별 NA 개수 확인
df.isnull().sum()

# row 단위 NA 개수 확인(axis = 0은 컬럼 기준이며 디폴트값임)
#df.isnull().sum(axis = 1)

# NAN에 해당하는 값을 임의의 다른 값으로 대체
df[['Product_Category_2', 'Product_Category_3']] = df[['Product_Category_2', 'Product_Category_3']].fillna(0)

# NA 값이 대체된 것을 확인
df.isnull().sum()

# Product_categry_1, 2, 3 변수의 값을 다 더한 Product_all 변수를 생성하여 추가

df[['Product_all']] = df['Product_Category_1'] + df['Product_Category_2'] + df['Product_Category_3']


# User_ID를 character 변수로, Occupation, Marital_Status, Product_Category_1,  Product_Category_2, Product_Category_3 변수를 범주형 변수로 변환


# 범주형 변수인 Gender, Age, City_Category, Stay_In_Current_City_Years를 더미변수로 변환해서 BlackFriday 데이터에 추가

df[['Gender', 'Age']].head()

print(df.columns)

df2 = pd.get_dummies(df[['Gender', 'Age', 'City_Category', 'Stay_In_Current_City_Years']])

# cbind라고 생각하면 됨, 컬럼 연결
df3 = pd.concat([df, df2], axis = 1)

print(df3.columns)



Index(['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category',
       'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1',
       'Product_Category_2', 'Product_Category_3', 'Purchase', 'Product_all'],
      dtype='object')
Index(['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category',
       'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1',
       'Product_Category_2', 'Product_Category_3', 'Purchase', 'Product_all',
       'Gender_F', 'Gender_M', 'Age_0-17', 'Age_18-25', 'Age_26-35',
       'Age_36-45', 'Age_46-50', 'Age_51-55', 'Age_55+', 'City_Category_A',
       'City_Category_B', 'City_Category_C', 'Stay_In_Current_City_Years_0',
       'Stay_In_Current_City_Years_1', 'Stay_In_Current_City_Years_2',
       'Stay_In_Current_City_Years_3', 'Stay_In_Current_City_Years_4+'],
      dtype='object')


In [3]:
#---------------------------------------------------------------------------------------
# Q2) BlackFriday_1 데이터에서 User_ID, Product_ID, Gender, Age, City_Category, Stay_In_Current_City_Years, 
#     Product_all  변수를 제외한 나머지 변수들을 활용하여 kmeans 군집분석을 수행하고 해석하시오.
#     (범주형 변수는 수치형 변수로 변환하여 분석에 활용하시오.)
#     그리고 최적의 군집 개수를 찾아서 만일 군집 개수가 이전 결과와 다르면 분석을 다시 수행하여 해석하시오.
#---------------------------------------------------------------------------------------

df4 = df3[df3.columns[~df3.columns.isin(['User_ID', 'Product_ID', 'Gender', 'Age', 'City_Category', 'Stay_In_Current_City_Years', 'Product_all'])]]

print(df4.columns)
print(df4.shape) # (537577, 23)

from sklearn.metrics.cluster import silhouette_score
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans

#model = KMeans(n_clusters = 3)

# 오래 걸릴 경우, 아래 미니 배치로 수행
model = MiniBatchKMeans(n_clusters = 3)
model.fit(df4)

print(model.labels_)

#print(f"실루엣 계수 = {silhouette_score(df4, model.labels_)}")




Index(['Occupation', 'Marital_Status', 'Product_Category_1',
       'Product_Category_2', 'Product_Category_3', 'Purchase', 'Gender_F',
       'Gender_M', 'Age_0-17', 'Age_18-25', 'Age_26-35', 'Age_36-45',
       'Age_46-50', 'Age_51-55', 'Age_55+', 'City_Category_A',
       'City_Category_B', 'City_Category_C', 'Stay_In_Current_City_Years_0',
       'Stay_In_Current_City_Years_1', 'Stay_In_Current_City_Years_2',
       'Stay_In_Current_City_Years_3', 'Stay_In_Current_City_Years_4+'],
      dtype='object')
(537577, 23)
[2 1 0 ... 2 0 0]


In [9]:
#---------------------------------------------------------------------------------------
# Q3) 최적의 군집 개수를 찾아 재수행한 군집분석의 결과로 분류된 군집번호를 원본 데이터인 BlackFriday데이터의 
#     각 행에 맞게 labelling하여 clust라는 변수로 저장하고 csv 파일로 출력하시오.
#     마지막으로 clust별 Gender, Age, Purchase으로 특성을 파악하시오.
#---------------------------------------------------------------------------------------

df[['clust']] = model.labels_

df.head()

# index => 인덱스 여부, header => 헤더 여부
df.to_csv('./output/20211031-2.csv', index = False)




